🔗 [Back to Table of Contents](https://github.com/najaeda/najaeda-tutorials#-table-of-contents)
# Chapter 1: Getting Started with najaeda

Welcome to **najaeda**! This guide will help you quickly install and start using the `najaeda` Python package to accelerate your EDA workflows.

📚 **API Documentation:**  
You can find the full API reference at [najaeda.readthedocs.io](https://najaeda.readthedocs.io/en/latest/)

💬 **Feedback & Contributions:**  
If you encounter bugs, have feature requests, or suggestions for improvement, please open an issue in our [GitHub issue tracker](https://github.com/najaeda/naja/issues).

⭐ **Support the Project:**  
If you find this project useful, consider [starring the repository](https://github.com/najaeda/naja) to show your support and help others discover it.

📧 **Contact Us:**  
For any questions or direct communication, feel free to reach out at [contact@keplertech.io](mailto:contact@keplertech.io)

## Introduction

**najaeda** is a powerful yet lightweight framework that empowers both software and hardware developers to efficiently explore, manipulate, and prototype electronic design automation (EDA) workflows.

Designed with flexibility and usability in mind, `najaeda` enables you to:

### Explore Netlists with Ease

- Intuitively navigate complex netlist hierarchies and connectivity.
- Inspect your design at various levels of abstraction:
  - Bit-level and bus-level granularity.
  - Instance-by-instance views or fully flattened representations down to primitives.
  - Focused per-instance connectivity or holistic equipotential (net-driven) views.

### Perform ECO (Engineering Change Order) Transformations

- Apply and manage design modifications smoothly.
- Accelerate the integration of late-stage design changes with confidence and control.

### Prototype EDA Concepts Rapidly

- Leverage an intuitive and expressive Python API.
- Experiment with new ideas, workflows, or data models in minutes, not days.

### Build Custom EDA Tools

- Develop high-performance, task-specific tools without depending on expensive, proprietary EDA suites.
- Solve domain-specific challenges with code that’s lean, readable, and maintainable.

## Installation

First, let's get the needed files for this tutorial repository to access the tutorials files.

In [10]:
!git clone https://github.com/najaeda/najaeda-tutorials.git

fatal : le chemin de destination 'najaeda-tutorials' existe déjà et n'est pas un répertoire vide.


To install `najaeda`, simply use pip:

In [11]:
pip install najaeda

zsh:1: no such file or directory: /Users/xtof/WORK/najaeda-tutorials/.venv/bin/python
Note: you may need to restart the kernel to use updated packages.


## Import najaeda
Now we will import najaeda needed package:

In [ ]:
from najaeda import netlist

## Load a verilog design
You can now load a gate level netlist using the loaded library.

In [ ]:
top = netlist.load_verilog(['najaeda-tutorials/benchmarks/verilog/adder/fulladder.v'])

## Explore the design
We can now start browsing some characteristics of the design.

In [ ]:
print(f'The top is named {top.get_name()}')
print(f'{top} number of input terms: {top.count_input_terms()}')
print(f'{top} number of output terms: {top.count_output_terms()}')

### Explore a top Terminal
Let's start exploring a specific object. The top 'A' terminal.

In [ ]:
a = top.get_term('a')
print(a)

### Explore a connected net


In [ ]:
aNet = a.get_net()
print(aNet)
for term in aNet.get_terms:
    print(term)